In [ ]:
import pandas as pd
import plotnine as p9
from pathlib import Path
from scipy import stats
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotnine as p9

import skmisc
from matplotlib.patches import Ellipse
from sklearn.decomposition import PCA

In [ ]:
def invnorm(x):
    return stats.norm.ppf((x.rank() - 0.5)/x.count())

In [ ]:
results_dir = "../../data/processed/results"
sample = 'dnaid2023'


In [ ]:
#TV5490A: library_13_1
counts_file = "/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/"\
              "data/processed/results/dnaid2023/dnaid2023_counts_TV5490A.txt"

mapping_file = "/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/"\
                "data/processed/results/dnaid2023/dnaid2023_features_TV5490A.txt"

results_file = "/nfs/nas22/fs2202/biol_micro_bioinf_nccr/hardt/nguyenb/tnseq/"\
                "data/processed/results/dnaid2023/dnaid2023_results_TV5490A.txt"

omapping_file = "/nfs/cds-peta/exports/biol_micro_cds_gr_sunagawa/scratch/"\
                "chris/hardt/nguyenb/tnseq_mapping_2031/scratch/library_13_1/barcode_map.txt"

In [ ]:
old_map = pd.read_table(omapping_file, header=None)
df = (pd.read_csv(counts_file, sep=" ")
    .drop(['Position', 'Element', 'Strand', 'Feature', 'ShortName' ], axis=1))
features = pd.read_csv(mapping_file, sep=" ")
results = pd.read_csv(results_file, sep=" ")

In [ ]:
df2 = df.apply(lambda x: x/x.sum())

In [ ]:
df2['al962-d1-feces'].hist(bins=50)

In [ ]:
df3 = df2.apply(invnorm)

In [ ]:
df3['al962-d1-feces'].hist(bins=50)

In [ ]:
df4 = np.log2(df2+1)

In [ ]:
df4['al962-d1-feces'].hist(bins=50)

In [ ]:
df2.corr()

In [ ]:
sns.set_style("white")
sns.set_context("notebook", font_scale=2.0)
g = sns.clustermap(df2.corr(), linewidths=0.5, linecolor='black',figsize=(15,15),vmin=0.6, vmax=1,
            cbar_kws={'label': 'Correlation Coefficient'})

In [ ]:
g = sns.clustermap(df.corr(), linewidths=0.5, linecolor='black',figsize=(15,15),vmin=0.6, vmax=1,
            cbar_kws={'label': 'Correlation Coefficient'})

In [ ]:
df1 = df.reset_index()
fdf = pd.melt(df1, id_vars = 'index')
expansion = fdf['variable'].str.split('-', expand=True)
fdf['mouse'], fdf['day'],fdf['organ'] = expansion[0], expansion[1], expansion[2]
meta = fdf.drop(['index', 'value'], axis=1).drop_duplicates().set_index('variable')
#fdf = df2.merge(features[['Feature', 'ShortName']], how='left', left_index=True, right_index=True)
#fdf = fdf.reset_index().rename({'index':'barcode'}, axis=1)

In [ ]:
meta

In [ ]:
def find_pc1_pc2(df, meta):
    df = df.T
    pca = PCA(n_components=2)
    principalComponents = pca.fit_transform(df)
    pDf = (pd.DataFrame(data=principalComponents, columns=['PC1', 'PC2'])
           .set_index(df.index))
    pc1_var = round(pca.explained_variance_ratio_[0] * 100, 2)
    pc2_var = round(pca.explained_variance_ratio_[1] * 100, 2)
    pDf2 = pDf.merge(meta, left_index=True, right_index=True)
    return pDf2, pc1_var, pc2_var

In [ ]:
pDF, pc1, pc2, = find_pc1_pc2(df3, meta)

In [ ]:
pDF, pc1, pc2, = find_pc1_pc2(df3, meta)

In [ ]:
pDF

In [ ]:
plotPCA(pDF, pc1, pc2, colorby='day', nameby='mouse', col=['blue', 'green', 'red']);


In [ ]:
"""
Plotting PCA elipses:
__author__:
"""

def plot_point_cov(points, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma ellipse based on the mean and covariance of a point
    "cloud" (points, an Nx2 array).
    Parameters
    ----------
        points : An Nx2 array of the data points.
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """
    pos = points.mean(axis=0)
    cov = np.cov(points, rowvar=False)
    return plot_cov_ellipse(cov, pos, nstd, ax, **kwargs)


def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """
    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:,order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:,0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta, **kwargs)

    ax.add_artist(ellip)
    return ellip
#____________________________________________________




def plotPCA(pDf, pc1_var, pc2_var, colorby, col, nameby="", el=False):
    sns.set_style("ticks")
    sns.set_context("notebook", font_scale=2.2)
    group = pDf[colorby].unique()
    assert len(group) <= len(col)
    fig = plt.figure(figsize=(8, 8))
    for g, c in zip(group, col):
        df = pDf[pDf[colorby] == g]
        x, y = df[["PC1"]].values, df[["PC2"]].values
        ax = plt.scatter(x, y, c=c, s=150, label=g)
        if el:
            pts = np.asarray([[float(a), float(b)] for a, b in zip(x, y)])
            plot_point_cov(pts, nstd=2, alpha=0.1, color=c)
        if nameby:
            labels = df[nameby]
            for label, pc1, pc2 in zip(labels, x, y):
                plt.annotate(label, xy=(pc1, pc2), xytext=(-5, 7), textcoords="offset points",fontsize=14)
        plt.xlabel('Principal Component 1, {} %'.format(pc1_var), )
        plt.ylabel('Principal Component 2, {} %'.format(pc2_var), )
        #plt.xticks(fontsize=16)
        #plt.yticks(fontsize=16)
        plt.legend(frameon=True)
    return fig

In [ ]:
controls = pd.read_table('../../data/metadata/controls.txt', header=None, 
                        names = ['DN', 'BC', 'phenotype', 'conc']).set_index('BC')
controls = controls.merge(df, how='left', left_index=True, right_index=True).fillna(0)

controls = controls.melt(id_vars=['DN', 'phenotype', 'conc']).rename({'variable':'sample', 'value':'counts'}, axis=1)
controls.head()

In [ ]:
p9.options.figure_size = (15, 15)
(p9.ggplot(controls[controls.phenotype == 'wt'], p9.aes(x='conc', y='counts'))
 #+ p9.geom_line()
 + p9.geom_point()
 +p9.geom_smooth(method = "lm")
 + p9.theme_classic()
 + p9.ylab("Count")
 + p9.xlab("Conc")
 + p9.scale_y_log10()
 + p9.scale_x_log10()
 + p9.facet_wrap('~ sample'))

In [ ]:
controls['logConc'] = np.log10(controls['conc'])
controls['logCounts'] = np.log10(controls['counts'])
controls.groupby(['phenotype', 'sample'])[['conc', 'counts']].corr().loc['hyb']

In [ ]:
p9.options.figure_size = (15, 15)
(p9.ggplot(controls[controls.phenotype == 'hyb'], p9.aes(x='conc', y='counts'))
 #+ p9.geom_line()
 + p9.geom_point()
 +p9.geom_smooth(method = "lm")
 + p9.theme_classic()
 + p9.ylab("Count")
 + p9.xlab("Conc")
 + p9.scale_y_log10()
 + p9.scale_x_log10()
 + p9.facet_wrap('~ sample'))

In [ ]:
wt_corr = controls.groupby(['phenotype', 'sample'])[['logConc', 'logCounts']].corr().loc['wt']
wt_corr = wt_corr.reset_index()
wt_corr = wt_corr[wt_corr['level_1'] == 'logCounts'].drop(['level_1','logCounts'], axis=1)
#wt_corr = wt_corr.stack()
#wt_corr.head()
#wt_corr = wt_corr[wt_corr.index.get_level_values(0) != wt_corr.index.get_level_values(1)]
#wt_corr.index = wt_corr.index.map('_'.join)
#wt_corr = wt_corr.to_frame().drop_duplicates()

In [ ]:
wt_corr.hist(bins=20)

In [ ]:
wt_corr = wt_corr.set_index('sample')
wt_corr

In [ ]:
prop_corr = df2.corr()[['inoculum-d0-inoculum']]
#prop_corr = prop_corr.merge(wt_corr, left_index=True, right_index=True)
prop_corr.hist(bins=20)

In [ ]:
sns.set_context("notebook", font_scale=2.4)
fig = plt.figure(figsize=(14,12))
plt.yscale('log')
#plt.xlim(-8, 8)
plt.rcParams["lines.markersize"] = 10.0
plt.scatter(results.d1_zscore, results.d1_padj, color='green', alpha=0.5, label="Day 1")
plt.scatter(results.d2_zscore, results.d2_padj, color='red', alpha = 0.5, label="Day 2")
#plt.scatter(aa_transp_df.log2FoldChange, aa_transp_df.padj, color="#74d600",  label="Amino Acid Transporters")
#plt.scatter(sugar_aa_met_df.log2FoldChange, sugar_aa_met_df.padj, color=col5, 
#            label="Sugar Catabolism/Amino Acid Biosynthesis")
#plt.scatter(sugar_tt_df.log2FoldChange, sugar_tt_df.padj, color="#00ced1",  label="Sugar Transporters")

#plt.axvline(-2, color='k', linestyle="--",)
#plt.axvline(2, color='k', linestyle="--")
plt.axhline(0.05, color='k', linestyle="--")
plt.xlabel("Log$_2$(Fold Change)", fontsize=36)
plt.ylabel("Adjusted $\itP$ value", fontsize=36)
plt.gca().invert_yaxis()

#plt.legend(markerscale=1.5, frameon=True, facecolor="white", framealpha=0.8, fontsize=24)

#plt.legend(markerscale=1.5, frameon=True, facecolor="white", framealpha=0.5)

In [ ]:
results.sort_values(by='d1_padj')

In [ ]:
pos_bias = df.merge(features, how='left', left_index =True, right_index = True)
pos_bias = pos_bias[pos_bias.Element == 'FQ312003.1'].sort_values(by='Position')
pos_bias['Position'] = pos_bias['Position']/1000000
pos_bias = pos_bias[pos_bias.sum(axis=1) > 10]

In [ ]:
p9.options.figure_size = (10, 10)
(p9.ggplot(pos_bias, p9.aes(x='Position', y='inoculum-d0-inoculum'))
 #+ p9.geom_line()
 + p9.geom_point()
 + p9.geom_segment(p9.aes(x='Position', xend='Position', y='inoculum-d0-inoculum', yend=0), alpha=0.5)
 + p9.theme_classic()
 + p9.geom_smooth(method='loess', color='red')
#  + p9.ylab("Count")
#  + p9.xlab("Conc")
  + p9.scale_y_log10()
)
 #+ p9.scale_color_distiller(palette=1, ))
#  + p9.facet_wrap('~ sample'))

In [ ]:
p9.options.figure_size = (10, 10)
(p9.ggplot(pos_bias, p9.aes(x='Position', y='al966-d1-feces'))
 #+ p9.geom_line()
 + p9.geom_point()
 + p9.geom_segment(p9.aes(x='Position', xend='Position', y='al966-d1-feces', yend=0), alpha=0.5)
 + p9.theme_classic()
 + p9.geom_smooth(method='loess', color='red')
#  + p9.ylab("Count")
#  + p9.xlab("Conc")
  + p9.scale_y_log10()
 #+ p9.scale_color_distiller(palette=1, ))
#  + p9.facet_wrap('~ sample'))
)

In [ ]:
p9.options.figure_size = (10, 10)
(p9.ggplot(pos_bias, p9.aes(x='Position', y='al966-d2-feces'))
 #+ p9.geom_line()
 + p9.geom_point()
 + p9.geom_segment(p9.aes(x='Position', xend='Position', y='al966-d2-feces', yend=0), alpha=0.5)
 + p9.theme_classic()
 + p9.geom_smooth(method='loess', color='red')
#  + p9.ylab("Count")
#  + p9.xlab("Conc")
  + p9.scale_y_log10()
)
 #+ p9.scale_color_distiller(palette=1, ))
#  + p9.facet_wrap('~ sample'))

In [ ]:
(p9.ggplot(geneInfo, p9.aes('Feature'))
 + p9.geom_histogram(bins=50, fill='#428bca')
 + p9.theme_classic()
 + p9.ylab("Count")
 + p9.xlab("# of barcodes per feature"))

In [ ]:
c1 = abs(results['d1_zscore']) > 2
c2 = results['d1_padj']< 0.05
results['d1_hit'] = c1 & c2
results['d1_-log10Pval'] = -np.log10(results.d1_padj)

In [ ]:

(p9.ggplot(results, p9.aes(x = 'd1_zscore', y = 'd1_-log10Pval', fill='d1_hit')) +
 p9.geom_point(size = 2, shape = 'o', colour = "grey")+
 p9.labs(x = 'Z-Score', y = '-log10(Pval)')+
 p9.scale_x_continuous(limits = [-12, 6], breaks = range(-12, 6, 2)) + 
 p9.scale_y_continuous(limits = [0, 25], breaks = range(0, 26, 5))+
 p9.ggtitle("Title")+
 p9.theme(
    plot_title = p9.element_text(family = "Arial", size = 11, hjust = 0), # Title size and font.
    
    axis_text = p9.element_text(family = "Arial", size = 10), # Size and font of x and y values.
    axis_title = p9.element_text(family = "Arial", size = 10), # Size and font of x and y axes.
    panel_border = p9.element_rect(colour = "black", fill = np.nan, size = 1), # Black border around the plot area.
    axis_ticks = p9.element_line(colour = "black", size = 1), # Style of x and y ticks.
    legend_position = "none"
  )+
   p9.geom_label(p9.aes(label=label), label_size=0.01, nudge_x=2,))#, adjust_text={'expand_points': (1.5, 1.5), 'arrowprops': {'arrowstyle': '-'}}))
  #p9.scale_fill_manual(breaks = ["blue", "red"], values = ["deepskyblue3", "firebrick1"]))



In [ ]:



  #geom_hline(yintercept = 1.30103, colour = "black", linetype = "dashed", size = 0.75) + # Horizontal significance cut-off line.
  #geom_vline(xintercept = 0.584963, colour = "black", linetype = "dashed", size = 0.75) + # Vertical significance cut-off line (+).
  # geom_vline (xintercept = -0.584963, colour = "black", linetype = "dashed", size = 0.75) #Vertical significance cut-off line (-)

  #scale_fill_manual(breaks = c("blue", "red"), values = c("deepskyblue3", "firebrick1")) + # Costum colors of data points based on "PursFur" column.

  #geom_text(aes(label = ifelse(PursFur == 1, as.character(Protein.ID), "")), hjust = 0, vjust = -0.25) + # Add identifiers to a subset of data points.

  #annotate("text", x = 2.9, y = 1.45, label = "P = 0.05", size = 4, fontface = "bold") + # Label to horizontal cut-off line.
  #annotate("text", x = 0.68, y = 5.9, label = "1.5-fold", size = 4, fontface = "bold", srt = 90) + # Label to vertical cut-off line.
  #annotate("rect", xmin = 0.584963, xmax = 3, ymin = 1.30103, ymax = 6, alpha = .2)) # Shade plot subregion.

In [ ]:
results.loc['gene-SL1344_RS24690']['d1_hit']

In [ ]:
label = [results.loc[i]['short_name'] if results.loc[i]['d1_hit'] else '' for i in results.index  ]

In [ ]:
label